In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import joblib
emb_matrix_train = joblib.load("/content/drive/MyDrive/absita_comfort_embedd_128_768_alberto_train.joblib")
wmal_matrix_train = joblib.load("/content/drive/MyDrive/wmal_train_comfort_128.joblib")

In [3]:
import pandas as pd
dataset_DF = pd.read_csv("/content/drive/MyDrive/comfort_train_processed.csv")

In [4]:
from sklearn.preprocessing import OneHotEncoder
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# define example
data = dataset_DF["neg"]
values = array(data)
print(values)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
# invert first example
inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
print(inverted)

[1 0 0 ... 1 1 1]
[1 0 0 ... 1 1 1]
[[0. 1.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[1]


In [4]:
print(len(onehot_encoded))

2670


In [5]:
from keras.layers import Input, Dense, LSTM, Bidirectional, TimeDistributed, Flatten, Concatenate
!pip install keras-self-attention

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for keras-self-attention: filename=keras_self_attention-0.51.0-py3-none-any.whl size=18912 sha256=3b307263c9d11f8b12d4c4a070af86974b29ea24d16bd566b7f114f39c211966
  Stored in directory: /root/.cache/pip/wheels/95/b1/a8/5ee00cc137940b2f6fa198212e8f45d813d0e0d9c3a04035a3
Successfully built keras-self-attention


In [6]:
from keras_self_attention import SeqWeightedAttention

In [7]:
import keras
input_emb = Input(shape=(128,768))
input_wmal = Input(shape=(128,))

#Bi-LSTM
bi = Bidirectional(LSTM(32, return_sequences=True))(input_emb)
att = SeqWeightedAttention()(bi)
flat = Flatten()(att)

#WMAL 
d1 = Dense(64) (input_wmal)

#Concat
concat = Concatenate(axis=1)([flat, d1])

d2 = Dense(32) (concat)
output = Dense(2,activation="softmax") (d2)

m = keras.Model(inputs=[input_emb,input_wmal], outputs=output)
m.compile(loss="categorical_crossentropy",optimizer='adam',metrics=["accuracy"])

In [ ]:
m.summary()

In [ ]:
m.fit([emb_matrix_train,wmal_matrix_train],onehot_encoded,batch_size=128, validation_split=0.15, epochs=10)

In [9]:
emb_matrix_test = joblib.load("/content/drive/MyDrive/absita_comfort_embedd_128_768_alberto_test.joblib")
dataset_test = pd.read_csv("/content/drive/MyDrive/comfort_test_processed.csv")
wmal_matrix_test = joblib.load("/content/drive/MyDrive/wmal_test_comfort_128.joblib")

In [10]:
# define example
data = dataset_test["neg"]
values = array(data)
print(values)
# integer encode
integer_encoded_test = label_encoder.transform(values)
print(integer_encoded_test)
# binary encode
integer_encoded_test = integer_encoded_test.reshape(len(integer_encoded_test), 1)
onehot_encoded_test = onehot_encoder.transform(integer_encoded_test)
print(onehot_encoded_test)

[1 1 1 ... 0 0 1]
[1 1 1 ... 0 0 1]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 0.]
 [1. 0.]
 [0. 1.]]


In [11]:
predictions = m.predict([emb_matrix_test,wmal_matrix_test])

In [ ]:
print(predictions)

In [13]:
len(predictions)

1171

In [12]:
import numpy as np
predictions_vec = np.zeros((1171,4))

In [13]:
i = 0
confidences= []
for item in predictions:
  index = np.argmax(item);
  confidences.append(item[index])
  predictions_vec[i][index] = 1
  i = i+1

In [14]:
labels_to_eval = []
for vec in predictions_vec:
  inverted = label_encoder.inverse_transform([argmax(vec)])[0]
  labels_to_eval.append(inverted)

In [ ]:
confidences

In [ ]:
len(labels_to_eval)

In [19]:
dataset_test["predicted_label_neg"] = labels_to_eval
dataset_test["confidence_label_neg"] = confidences

In [ ]:
dataset_test["neg"]

In [15]:
!pip install scikit-learn

from sklearn.metrics import classification_report
print(classification_report(dataset_test["neg"],labels_to_eval,digits=5))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
              precision    recall  f1-score   support

           0    0.94495   0.84774   0.89371       486
           1    0.89932   0.96496   0.93099       685

    accuracy                        0.91631      1171
   macro avg    0.92214   0.90635   0.91235      1171
weighted avg    0.91826   0.91631   0.91552      1171



In [ ]:
dataset_test

In [16]:
m.save("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/models/model_1_neg_absita.h5")

In [ ]:
dataset_test.to_csv("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/models/predictions_AGRITREND_neg_full.csv");